In [200]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.preprocessing import Normalizer, MaxAbsScaler
from sklearn.pipeline import make_pipeline

In [201]:
artists = pd.read_csv('C:\\Users\\Deepak\\Downloads\\artists.csv', header = None, names = ['Artists'])
samples = pd.read_csv('C:\\Users\\Deepak\\Downloads\\scrobbler-small-sample.csv')

In [202]:
artists.shape

(111, 1)

In [203]:
artists.head()

,Artists
0,Massive Attack
1,Sublime
2,Beastie Boys
3,Neil Young
4,Dead Kennedys


In [204]:
samples.shape

(2894, 3)

In [205]:
samples.head()

,user_offset,artist_offset,playcount
0,1,79,58
1,1,84,80
2,1,86,317
3,1,89,64
4,1,96,159


In [206]:
samples = samples.pivot_table(index = 'artist_offset', columns = 'user_offset', values = 'playcount')
samples.head()

user_offset,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
artist_offset,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,128.0,211.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,270.0,NaN,105.0,97.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:
samples = samples.replace(np.NaN, 0)

In [208]:
samples = samples.reset_index()
samples['artist_offset'] = artists
samples.head()

user_offset,artist_offset,0,1,2,3,4,5,6,7,8,...,490,491,492,493,494,495,496,497,498,499
0,Massive Attack,0.0,0.0,105.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Sublime,128.0,211.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,270.0,0.0,105.0,97.0,0.0,0.0,0.0
2,Beastie Boys,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Neil Young,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Dead Kennedys,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [209]:
samples = samples.rename(columns = {'artist_offset':'Artists'})

In [210]:
samples.head()

user_offset,Artists,0,1,2,3,4,5,6,7,8,...,490,491,492,493,494,495,496,497,498,499
0,Massive Attack,0.0,0.0,105.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Sublime,128.0,211.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,270.0,0.0,105.0,97.0,0.0,0.0,0.0
2,Beastie Boys,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Neil Young,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Dead Kennedys,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [211]:
articles = samples.iloc[:,1:]
articles.head()

user_offset,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.0,0.0,105.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,128.0,211.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,270.0,0.0,105.0,97.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [212]:
scaler = MaxAbsScaler()
nmf = NMF(n_components = 20)
normalizer = Normalizer()
pipeline = make_pipeline(scaler, nmf, normalizer)
norm_features = pipeline.fit_transform(articles)
norm_features.shape

(111, 20)

In [213]:
norm_features

array([[0.        , 0.05709646, 0.        , ..., 0.        , 0.24512865,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.18894523, ..., 0.29664213, 0.56425941,
        0.04068795],
       [0.        , 0.00563782, 0.00646817, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.05611861, 0.        , ..., 0.        , 0.36163446,
        0.60353564]])

In [214]:
df = pd.DataFrame(norm_features, index = samples['Artists'])
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Artists,,,,,,,,,,,,,,,,,,,,
Massive Attack,0.00000,0.057096,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017597,0.248608,0.430656,0.296666,0.775281,0.0,0.000000,0.000000,0.000000,0.245129,0.0
Sublime,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.0
Beastie Boys,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
Neil Young,0.26479,0.000000,0.0,0.055263,0.0,0.0,0.0,0.009024,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.960273,0.065337,0.000000,0.018935,0.0
Dead Kennedys,0.00000,0.000000,0.0,0.583660,0.0,0.0,0.0,0.142908,0.0,0.733425,0.000000,0.000000,0.138940,0.000000,0.0,0.077106,0.000000,0.275238,0.000000,0.0


In [215]:
artist = df.loc['Bruce Springsteen']
similarities = df.dot(artist)
print(similarities.nlargest())

Artists
Bruce Springsteen    1.000000
Neil Young           0.958831
Leonard Cohen        0.916741
Van Morrison         0.883516
Bob Dylan            0.866162
dtype: float64
